In [1]:
! pip install numpy faiss-cpu PyMuPDF docx2txt langchain tiktoken

In [2]:
! pip install openai==0.28

In [15]:
import os
import pickle
import fitz
import docx2txt
import json
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


class General_Fields_Helper:

    def list_files_local_dir(self, dirpath, pickle_files=False):
        """Returns a list of files with their paths in the local directory"""

        if pickle_files:
            return [os.path.join(dirpath, file) for file in os.listdir(dirpath)]
        else:
            return [os.path.join(dirpath, file) for file in os.listdir(dirpath) if not file.endswith('pkl')]

    def get_document_text(self, filepath):
        """Returns the text extracted from a document"""
        if os.path.isfile(filepath):
            with open(filepath, 'rb') as file:
                content = file.read()

            # Retrieve the text from the documents uploaded in pdf format
            if filepath.endswith('.pdf'):
                pdf = content
                reader = fitz.open(stream=pdf, filetype='pdf')
                data = ""
                for page in reader:
                    data += page.get_text()
                reader.close()
                assert data != "", "There is no selectable text in the PDF file"

            # Retrieve the text from the documents uploaded in docx format
            elif filepath.endswith('.docx'):
                doc = content
                data = docx2txt.process(doc)
                assert data != "", "There is no selectable text in the DOCX file"

            elif filepath.endswith('.txt'):
                return content.decode('utf-8')
            elif filepath.endswith('.json'):
                json_str = content.decode('utf-8')
                data = json.loads(json_str)
                return data['results']['transcripts'][0]['transcript']
            else:
                raise ValueError(f"Unsupported file format for {filepath}")
        else:
            print(f"{filepath} is not a file. Ignoring.")
            return None

    def split_into_chunks(self, text):
        """Splits the text into chunks of specific token length and returns a list of those chunks"""

        try:
            # if text length more than 500 words
            text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)
            texts = text_splitter.split_text(text)
            return texts
        except:
            # if text length less than 500 words
            return [text]

    def read_embeddings(self, dirpath):
        """Reads the vector store from the directory and returns the faiss object"""

        vecstorepath = os.path.join(dirpath, "vecstore.pkl")
        with open(vecstorepath, 'rb') as file:
            vecstore = pickle.load(file)
        return vecstore

    def update_embeddings(self, texts, dirpath, key , save=True):
      vecstorepath = os.path.join(dirpath, "vecstore.pkl")
      if os.path.exists(vecstorepath):
        with open(vecstorepath, 'rb') as file:
          vecstore = pickle.load(file)
          vecstore.add_texts(texts)
      else:
        embeddings = OpenAIEmbeddings(openai_api_key=key)
        vecstore = FAISS.from_texts(texts, embeddings)
      if save:
        with open(vecstorepath, 'wb') as file:
           pickle.dump(vecstore, file)
      return vecstore
    def top_similar_docs(self, user_query, store):
        """Returns the the top similar documents to the user query, using the vector store"""

        docs = store.similarity_search(query=user_query)
        return docs



In [16]:
! mkdir docsresume

mkdir: cannot create directory ‘docsresume’: File exists


In [17]:
ls-a "/content/docsresume/"

 ./   ../  'output (1).txt'


In [18]:
cat "/content/docsresume/vecstore.pkl"

cat: /content/docsresume/vecstore.pkl: No such file or directory


In [19]:
from apikeys import chatgpt

# print(chatgpt())

# Instantiate the helper class
helper = General_Fields_Helper()

# Define the directory path
dirpath = "docsresume"

# List all files in the 'docs' directory
files = helper.list_files_local_dir(dirpath)

# Extract texts from all files and store in a list
all_texts = []
for file in files:
    text = helper.get_document_text(file)
    all_texts.append(text)

# Assuming each file contains a large amount of text, we'll split them into chunks
# This is especially useful if your embeddings or vector store has a limitation on the text size
chunked_texts = []
for text in all_texts:
    chunked_texts.extend(helper.split_into_chunks(text))

apikey = chatgpt()

# Update the embeddings with the chunked_texts. This will create or update the vecstore.pkl in the 'docs' directory
vecstore = helper.update_embeddings(chunked_texts, dirpath , apikey)

# Now, to get the top similar docs for a given query:
user_query = "You have been given a resume , What is the name of Applicant in the resume in one word  ?"  # replace with your actual query
top_docs = helper.top_similar_docs(user_query, vecstore)
print(top_docs)

[Document(page_content="CURRICULUM VITAE\nCurrent:- 35k, Expectation:- 45K\n\nSantosh Kumar\n\nB.E.(Computer Engg.) Mob. No. +91  7266895785 ,+91 7530824177\n\nEmail: santoshkumar5dec@gmail.com \n\t\nPassport No. R7506800\n\nCAREER OBJECTIVE\nTo work with a high level of sincerity and hard work to achieve marked goals and to acquire the \nbest solution for fulfillment of organizational need.\nORGANISATIONAL EXPERIENCE:\n\nCurrently 8 Years 1 months Software Development Experience:  \nTotal  11 years 1 months experience (Development(Currently)  + Teaching(previous-3 years))\n\nRecently working with PelSoft Labs Pvt. Ltd. (New Delhi) as Software Developer from June-2016 to \ntill now\nWorked with DFH Technologies. Ltd.(Gurgaon-Haryana) as Software Developer from Jan-2016 to \n-June-2016\nWorked with GoDunia Infonet Pvt.Ltd.(Gurgaon-Haryana) as Software Developer from jan-2014 to \ndec-2015\nWorked with BMC Software Pvt. Ltd.(Pune-Maharastra) as R&D Technical Associate for 4 months\nGATE 